In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import gridspec
import warnings
from osgeo import gdal, osr
import numpy as np
import cv2
import icepyx as ipx
import os
import shutil
%matplotlib inline

# utility modules
import glob
import os
import sys
import re
from osgeo import gdal, osr
import geopandas as gpd
import pandas as pd
import rasterio
from rasterio.plot import show

# the usual suspects:
import numpy as np
import matplotlib.pyplot as plt

# specialty modules
import h5py
import pyproj

# modules you'll need if you're downloading the data:
import geopandas as gpd
from icepyx.core.visualization import Visualize
from typing import Dict, List, Tuple

In [ ]:
try:
    import pointCollection as pc
except Exception:
    !python3 -m pip install --user git+https://github.com/smithb/pointCollection.git
    import pointCollection as pc

In [ ]:
# First lets extract elevation data from a line shapefile from the REMA tile
# from: https://portailsig.org/content/python-utilisation-des-couches-vectorielles-et-matricielles-dans-une-perspective-geologique-.html

In [ ]:
from osgeo import gdal,ogr
# raster dem10m
# Site D
file = './Site D extended area.tif'
layer = gdal.Open(file)
gt = layer.GetGeoTransform()
bands = layer.RasterCount
print(bands)
print(gt)

In [ ]:
def Val_raster(x,y,layer,bands,gt):
    col=[]
    px = int((x - gt[0]) / gt[1])
    py =int((y - gt[3]) / gt[5])
    for j in range(bands):
        band = layer.GetRasterBand(j+1)
        data = band.ReadAsArray(px,py, 1, 1)
        col.append(data[0][0])
    return col

In [ ]:
# with a DEM (1 band)
px1 = int((x - gt[0]) / gt[1])
py1 = int((y - gt[3]) / gt[5])
# print(Val_raster(x,y,layer, bands,gt))
# elevation

In [ ]:
# First define our own color ramp for cmap
# Based from: # https://stackoverflow.com/questions/16834861/create-own-colormap-using-matplotlib-and-plot-color-scale (Can Sucuoglu 2nd Jun 2020 22:35)

In [ ]:
def make_Ramp( ramp_colors ): 
    from colour import Color
    import matplotlib.pyplot as plt
    import numpy as np
    from matplotlib.colors import LinearSegmentedColormap

    color_ramp = LinearSegmentedColormap.from_list( 'my_list', [ Color( c1 ).rgb for c1 in ramp_colors ] )
    plt.figure( figsize = (15,3))
    plt.imshow( [list(np.arange(0, len( ramp_colors ) , 0.1)) ] , interpolation='nearest', origin='lower', cmap= color_ramp )
    plt.xticks([])
    plt.yticks([])
    return color_ramp


custom_ramp = make_Ramp( ['#FFFFFF','#FFFFFF','#FFFFFF','#FFFFFF','#FFFFFF','#FFFFFF','#FFFFFF','#e5e5e5','#e5e5e5','#cccccc','#cccccc','#b2b2b2','#b2b2b2','#7f7f7f','#7f7f7f','#4c4c4c','#4c4c4c','#191919','#191919','#000000' ] ) 

In [ ]:
# As we are working with a DEM we need to produce a hillshade
# Based from:https://www.neonscience.org/resources/learning-hub/tutorials/create-hillshade-py
# Note scale factor not working which impact visual look later.

def raster2array(geotif_file):

    metadata = {}
    dataset = gdal.Open(geotif_file)
    metadata['array_rows'] = dataset.RasterYSize
    metadata['array_cols'] = dataset.RasterXSize
    metadata['bands'] = dataset.RasterCount
    metadata['driver'] = dataset.GetDriver().LongName
    metadata['projection'] = dataset.GetProjection()
    metadata['geotransform'] = dataset.GetGeoTransform()

    mapinfo = dataset.GetGeoTransform()
    metadata['pixelWidth'] = mapinfo[1]
    metadata['pixelHeight'] = mapinfo[5]

    metadata['ext_dict'] = {}
    metadata['ext_dict']['xMin'] = mapinfo[0]
    metadata['ext_dict']['xMax'] = mapinfo[0] + dataset.RasterXSize/mapinfo[1]
    metadata['ext_dict']['yMin'] = mapinfo[3] + dataset.RasterYSize/mapinfo[5]
    metadata['ext_dict']['yMax'] = mapinfo[3]

    metadata['extent'] = (metadata['ext_dict']['xMin'],metadata['ext_dict']['xMax'],
                          metadata['ext_dict']['yMin'],metadata['ext_dict']['yMax'])

    if metadata['bands'] == 1:
        raster = dataset.GetRasterBand(1)
        metadata['noDataValue'] = raster.GetNoDataValue()
        metadata['scaleFactor'] = raster.GetScale()

        # band statistics
        metadata['bandstats'] = {} #make a nested dictionary to store band stats in same
        stats = raster.GetStatistics(True,True)
        metadata['bandstats']['min'] = round(stats[0],2)
        metadata['bandstats']['max'] = round(stats[1],2)
        metadata['bandstats']['mean'] = round(stats[2],2)
        metadata['bandstats']['stdev'] = round(stats[3],2)

        array = dataset.GetRasterBand(1).ReadAsArray(0,0,metadata['array_cols'],metadata['array_rows']).astype(np.float)
        array[array==metadata['noDataValue']]=np.nan
#         array = array/metadata['scaleFactor']
        array = array[::-1] #inverse array because Python is column major
        return array, metadata

    elif metadata['bands'] > 1:
        print('More than one band ... need to modify function for case of multiple bands')

def array2raster(newRasterfn,rasterOrigin,pixelWidth,pixelHeight,array,epsg):

    cols = array.shape[1]
    rows = array.shape[0]
    originX = rasterOrigin[0]
    originY = rasterOrigin[1]

    driver = gdal.GetDriverByName('GTiff')
    outRaster = driver.Create(newRasterfn, cols, rows, 1, gdal.GDT_Byte)
    outRaster.SetGeoTransform((originX, pixelWidth, 0, originY, 0, pixelHeight))
    outband = outRaster.GetRasterBand(1)
    outband.WriteArray(array)
    outRasterSRS = osr.SpatialReference()
    outRasterSRS.ImportFromEPSG(epsg)
    outRaster.SetProjection(outRasterSRS.ExportToWkt())
    outband.FlushCache()

def plot_band_array(band_array,refl_extent,title,cbar_label,colormap='gist_earth',alpha=1):
    plt.imshow(band_array,extent=refl_extent,alpha=alpha);
    cbar = plt.colorbar(); plt.set_cmap(colormap);
    cbar.set_label(cbar_label,rotation=270,labelpad=20)
    plt.title(title); ax = plt.gca();
    ax.ticklabel_format(useOffset=False, style='plain') #do not use scientific notation #
    rotatexlabels = plt.setp(ax.get_xticklabels(),rotation=90) #rotate x tick labels 90 degree

def hillshade(array,azimuth,angle_altitude):
    azimuth = 360.0 - azimuth

    x, y = np.gradient(array)
    slope = np.pi/2. - np.arctan(np.sqrt(x*x + y*y))
    aspect = np.arctan2(-x, y)
    azimuthrad = azimuth*np.pi/180.
    altituderad = angle_altitude*np.pi/180.

    shaded = np.sin(altituderad)*np.sin(slope) + np.cos(altituderad)*np.cos(slope)*np.cos((azimuthrad - np.pi/2.) - aspect)

    return 255*(shaded + 1)/2

In [ ]:
# Now plot them
# # Site D
Site_D_teak_dtm_array, Site_D_teak_dtm_md = raster2array("./Site D extended area.tif")
Site_D_teak_hillshade_array = hillshade(Site_D_teak_dtm_array,45,20)
# Now the brightness and contrast need modifying so use cv2 for this.
alpha = 0.8
beta = -75
adjusted = cv2.convertScaleAbs(Site_D_teak_hillshade_array, alpha=alpha, beta=beta)

In [ ]:
# Now add elevations along same REMA profile from both CryoSat-2 and ICESat-2.

In [ ]:
# ICESat-2 data processing (ATL06)
def atl06_to_dict(filename, beam, field_dict=None, index=None, epsg=None):
    """
        Read selected datasets from an ATL06 file

        Input arguments:
            filename: ATl06 file to read
            beam: a string specifying which beam is to be read (ex: gt1l, gt1r, gt2l, etc)
            field_dict: A dictinary describing the fields to be read
                    keys give the group names to be read, 
                    entries are lists of datasets within the groups
            index: which entries in each field to read
            epsg: an EPSG code specifying a projection (see www.epsg.org).  Good choices are:
                for Greenland, 3413 (polar stereographic projection, with Greenland along the Y axis)
                for Antarctica, 3031 (polar stereographic projection, centered on the Pouth Pole)
        Output argument:
            D6: dictionary containing ATL06 data.  Each dataset in 
                dataset_dict has its own entry in D6.  Each dataset 
                in D6 contains a numpy array containing the 
                data
    """
    if field_dict is None:
        field_dict={None:['latitude','longitude','h_li', 'atl06_quality_summary'],\
                    'ground_track':['x_atc','y_atc'],\
                    'fit_statistics':['dh_fit_dx', 'dh_fit_dy']}
    D={}
    file_re=re.compile('ATL06_(?P<date>\d+)_(?P<rgt>\d\d\d\d)(?P<cycle>\d\d)(?P<region>\d\d)_(?P<release>\d\d\d)_(?P<version>\d\d).h5')
    with h5py.File(filename,'r') as h5f:
        for key in field_dict:
            for ds in field_dict[key]:
                if key is not None:
                    ds_name=beam+'/land_ice_segments/'+key+'/'+ds
                else:
                    ds_name=beam+'/land_ice_segments/'+ds
                if index is not None:
                    D[ds]=np.array(h5f[ds_name][index])
                else:
                    D[ds]=np.array(h5f[ds_name])
                if '_FillValue' in h5f[ds_name].attrs:
                    bad_vals=D[ds]==h5f[ds_name].attrs['_FillValue']
                    D[ds]=D[ds].astype(float)
                    D[ds][bad_vals]=np.NaN
    if epsg is not None:
        xy=np.array(pyproj.proj.Proj(epsg)(D['longitude'], D['latitude']))
        D['x']=xy[0,:].reshape(D['latitude'].shape)
        D['y']=xy[1,:].reshape(D['latitude'].shape)
    temp=file_re.search(filename)
    D['rgt']=int(temp['rgt'])
    D['cycle']=int(temp['cycle'])
    D['beam']=beam
    return D

In [ ]:
# find the matching ATL06 file

# read the data:
rgt="004"
cycle="01"

#34

ATL06_file=glob.glob(os.path.join('./ICESat-2 data Site D/',f'processed_ATL06*{rgt}_{cycle}*.h5'))[10] # 107
D6=atl06_to_dict(ATL06_file,'/gt2l', index=None, epsg=3031)

# plot the elevations on top of the previous axes.  You should be able to scroll up to the previous plot and see the ATL06 points.
fig=plt.figure(figsize=(22,12))
ax = fig.add_subplot(111)
ax.plot(D6['x_atc'], D6['h_li'],'r.', label='ATL06')
ax.set_xlabel('x_atc, m')
ax.set_ylabel('h, m')
ax.legend(prop={'size': 20},markerscale=3);

plt.savefig('./Site D/ICESat-2 images/ATL06 Data - Site D.png')
plt.show()

print("x: ", D6['x'],"y: ",D6['y'])
print(len(D6['h_li']))
print(len(D6['x_atc']))
print(len(D6['y_atc']))

In [ ]:
fig, (ax1) = plt.subplots(figsize=(16,16))

ax1.imshow(adjusted, cmap= custom_ramp,extent=[1980720.0, 2100920.0,-600312.0,-500064.0],origin='lower');
ax1=plt.gca(); ax1.ticklabel_format(useOffset=False, style='plain') #do not use scientific notation
rotatexlabels = plt.setp(ax1.get_xticklabels(),rotation=90) #rotate x tick labels 90 degrees
ax1.plot(D6['x'],D6['y'], linewidth=2, color ='firebrick', label='ICESAT-2 data transect')
# ax1.plot(CS2_df_v2['xx'],CS2_df_v2['yy'], linewidth=2, color ='blue', label='CS2 transect')
ax1.axis('image')
# ax2.legend(loc="upper right")
ax1.set_xlabel('Longitude')
ax1.set_ylabel('Latitude')
ax1.set_xlim(1980720, 2100920,)
ax1.set_ylim(-600312,-500064.0)

plt.savefig('./Site D/ICESat-2 images/ICESAT-2 Data map.png')
plt.show()

In [ ]:
# Makes sure REMA elevations along ICESat-2 flightpath are extracted 

In [ ]:
# Converting IS2 lat longs to distances
# Based from Github travisteague: https://github.com/travisteague/LatLongDistanceCalc_Sorting/blob/master/InternshipScript.py

import pandas as pd
from geopy import distance #caculate distances
from geopy import point #create points based of floating point lat & long and points based on string (N,S,E,W) 

IS2_list = [] #list to store the distances 
IS2_REMA_z_list=[] #List to storee calculated elevations at points
IS2_index_list = [] #list to store the indexs of the distances
IS2_REMA_index_list =[]
IS2_lat =[]
IS2_lon =[]
IS2_REMA_lat_lon = []
loc1 = (' -71.158607, 104.394020') #Coordinate of first lat and long (put latitude point first)


Reverse_lat = D6['latitude'][::-1].astype(str)
Reverse_lon = D6['longitude'][::-1].astype(str)
Reversed_IS2_elev = D6['h_li'][::-1]
zippedList =  list(zip(Reverse_lon, Reverse_lat, Reversed_IS2_elev))

IS2_df = pd.DataFrame(zippedList, columns = ['Longitude' , 'Latitude', 'IS2_elevation'])

In [ ]:
for i in range(len(IS2_df)): #loop throught data frame and compute the distances from the origin(loc1) to the specified location
      
    lat = IS2_df.at[i, 'Latitude']    
    longi = IS2_df.at[i, 'Longitude'] 
        
    IS2_lat.append(IS2_df.at[i, 'Latitude']) #grab the latitude from the index i in the dataframe
    IS2_lon.append(IS2_df.at[i, 'Longitude']) #grab the longitude from the index i in the dataframe
    
    pt = point.Point(lat + ',' + longi) #create a single point from the latitudes and longitudes
    IS2_REMA_lat_lon.append(pt)
    
    IS2_list.append(distance.distance(loc1, pt).meters) #calculate the distance from the origin to the point and append to the list    
    IS2_index_list.append(i) #append the index of the distance so they will match when joined with the originial dataframe 

In [ ]:
cols = IS2_df.select_dtypes(exclude=['float']).columns

IS2_df[cols] = IS2_df[cols].apply(pd.to_numeric, downcast='float', errors='coerce')
    
    
Reverse_lat_v2 = D6['latitude'][::-1].astype(float)
Reverse_lon_v2 = D6['longitude'][::-1].astype(float)
zippedList =  list(zip(Reverse_lon_v2, Reverse_lat_v2))
IS2_df_v2 = pd.DataFrame(zippedList, columns = ['Longitude' , 'Latitude',])


from pyproj import Proj
pp = Proj("+proj=stere +lat_0=-90 +lat_ts=-71 +lon_0=0 +k=1 +x_0=0 +y_0=0 +ellps=WGS84 +datum=WGS84 +units=m +no_defs")
xx, yy = pp(IS2_df_v2["Longitude"].values, IS2_df_v2["Latitude"].values)

print(xx)
print(yy)

zippedList =  list(zip(Reverse_lon_v2, Reverse_lat_v2, xx, yy))
IS2_df_v2 = pd.DataFrame(zippedList, columns = ['Longitude' , 'Latitude','xx','yy'])
print(IS2_df_v2.head)

In [ ]:
for i in range(len(IS2_df)):
    x = IS2_df_v2.at[i,'xx'] 
#     print('x: ',x)
    
    y = IS2_df_v2.at[i,'yy'] 
#     print('y: ',y)
    
    rasterx = int((x - gt[0]) / gt[1])  
    print('rasterx: ',rasterx)
    
    rastery = int((y - gt[3]) / gt[5])
    print('rastery: ',rastery)
    
    # only one band here   
#     IS2_REMA_z_list.append(Val_raster(x,y,layer, bands,gt))
    import re
    t =str(layer.GetRasterBand(1).ReadAsArray(rasterx,rastery, 1, 1))[1:-1]
    z = re.sub(r'\[', '', str(t)) 
    y = re.sub(r'\]','', z) 
    IS2_REMA_z_list.append(y)
    IS2_REMA_index_list.append(i)
    
print(IS2_REMA_z_list)

In [ ]:
dis_df = pd.DataFrame() #Create a dataframe of the distances
dis_df['Index'] = IS2_index_list
dis_df['Distance m'] = IS2_list
# dis_df['REMA Index'] = IS2_REMA_index_list
dis_df['REMA elevation'] = IS2_REMA_z_list

IS2_complete_df = IS2_df.join(dis_df.set_index('Index')) #join the original dataframe on the distances dataframe

In [ ]:
print(IS2_complete_df.head)
print(IS2_complete_df.dtypes)

In [ ]:
cols = IS2_complete_df.select_dtypes(exclude=['float']).columns

IS2_complete_df[cols] = IS2_complete_df[cols].apply(pd.to_numeric, downcast='float', errors='coerce')


IS2_complete_df['REMA elevation'].astype('float64')

print(IS2_complete_df)
print(IS2_complete_df.dtypes)

In [ ]:
# Now remove rows where REMA elevation = NaN due to tile not spanning this distance
IS2_complete_df = IS2_complete_df.dropna(subset = ["REMA elevation"])          # Apply dropna() function

index_vals = IS2_complete_df[IS2_complete_df['REMA elevation'] == 0].index

IS2_complete_df = IS2_complete_df.drop(index_vals)

print(IS2_complete_df)  

In [ ]:
fig, ax = plt.subplots(figsize=(16,8))
ax.plot(IS2_complete_df['Distance m'],IS2_complete_df['IS2_elevation'], label='ICESat-2 transect')
ax.set_xlabel('Distance along transect (m)')
ax.set_ylabel('Elevation (m)')
# ax.set_xlim(0,185000)
plt.legend()

plt.savefig('./Site D test ICESat-2 transect fig.jpg',dpi=600)
plt.show()

In [ ]:
# # Now Add CryoSat-2 data

In [ ]:
import netCDF4 as nc

cycle = "D001"

CS2_file=glob.glob(os.path.join('./CryoSat-2 Site D/',f'CS_LTA__SIR_SIN_2*{cycle}*.nc'))[8]
# D6=atl06_to_dict(ATL06_file,'/gt2l', index=None, epsg=3031)
print(CS2_file)


CS2_data = nc.Dataset(CS2_file)
print(CS2_data.variables)

In [ ]:
CS2_latitude = CS2_data['lat_poca_20_ku'][:]
print(CS2_latitude)
print(len(CS2_latitude))

CS2_longitude = CS2_data['lon_poca_20_ku'][:] 
print(CS2_longitude)
print(len(CS2_longitude))

CS2_time = CS2_data['time_20_ku']
print(CS2_time)
print(len(CS2_time))

CS2_elevation1 = CS2_data['height_1_20_ku'][:]
print(CS2_elevation1)
print(len(CS2_elevation1))

In [ ]:
import pandas as pd
from geopy import distance #caculate distances
from geopy import point #create points based of floating point lat & long and points based on string (N,S,E,W) 

CS2_list = [] #list to store the distances 
CS2_REMA_z_list=[] #List to storee calculated elevations at points
CS2_index_list = [] #list to store the indexs of the distances
CS2_REMA_index_list =[]
CS2_lat =[]
CS2_lon =[]
CS2_REMA_lat_lon = []
CS2loc1 = (' -71.158607, 104.394020') #Coordinate of first lat and long (put latitude point first)
 
CS2_longitude = CS2_data['lon_poca_20_ku'][::-1].astype(str)
CS2_latitude = CS2_data['lat_poca_20_ku'][::-1].astype(str)
CS2_elevation1 = CS2_data['height_1_20_ku'][::-1]
# CS2_elevation3 = CS2_data['height_3_20_ku'][::-1]

zippedList =  list(zip(CS2_longitude, CS2_latitude, CS2_elevation1))
CS2_df = pd.DataFrame(zippedList, columns = ['Longitude' , 'Latitude', 'CS2_elev_1'])

In [ ]:
for i in range(len(CS2_df)): #loop throught data frame and compute the distances from the origin(loc1) to the specified location
      
    lat = CS2_df.at[i, 'Latitude']  
#     print(lat)
    longi = CS2_df.at[i, 'Longitude']
#     print(longi)
        
        
    CS2_lat.append(CS2_df.at[i, 'Latitude']) #grab the latitude from the index i in the dataframe
    CS2_lon.append(CS2_df.at[i, 'Longitude']) #grab the longitude from the index i in the dataframe
    
    pt = point.Point(lat + ',' + longi) #create a single point from the latitudes and longitudes
    CS2_REMA_lat_lon.append(pt)
    
    CS2_list.append(distance.distance(CS2loc1, pt).meters) #calculate the distance from the origin to the point and append to the list    
    CS2_index_list.append(i) #append the index of the distance so they will match when joined with the originial dataframe 

In [ ]:
cols = CS2_df.select_dtypes(exclude=['float']).columns

CS2_df[cols] = CS2_df[cols].apply(pd.to_numeric, downcast='float', errors='coerce')
    
CS2_longitude_v2 = CS2_data['lon_poca_20_ku'][::-1].astype(float)
CS2_latitude_v2 = CS2_data['lat_poca_20_ku'][::-1].astype(float)

zippedList =  list(zip(CS2_longitude_v2, CS2_latitude_v2))
CS2_df_v2 = pd.DataFrame(zippedList, columns = ['Longitude' , 'Latitude',])


from pyproj import Proj
pp = Proj("+proj=stere +lat_0=-90 +lat_ts=-71 +lon_0=0 +k=1 +x_0=0 +y_0=0 +ellps=WGS84 +datum=WGS84 +units=m +no_defs")
xx, yy = pp(CS2_df_v2["Longitude"].values, CS2_df_v2["Latitude"].values)

print(xx)
print(yy)

zippedList =  list(zip(CS2_longitude_v2, CS2_latitude_v2, xx, yy))
CS2_df_v2 = pd.DataFrame(zippedList, columns = ['Longitude' , 'Latitude','xx','yy'])
print(CS2_df_v2.head)

In [ ]:
for i in range(len(CS2_df)):
    x = CS2_df_v2.at[i,'xx'] 
#     print(x)
    
    y = CS2_df_v2.at[i,'yy'] 
#     print(y)
    
    rasterx = int((x - gt[0]) / gt[1])    
#     print('rasterx: ',rasterx)
    
    rastery = int((y - gt[3]) / gt[5])
#     print('rastery: ',rastery)
    
    # only one band here   
#     IS2_REMA_z_list.append(Val_raster(x,y,layer, bands,gt))
    import re
    t =str(layer.GetRasterBand(1).ReadAsArray(rasterx,rastery, 1, 1))[1:-1]
    z = re.sub(r'\[', '', str(t)) 
    y = re.sub(r'\]','', z) 
    CS2_REMA_z_list.append(y)
    CS2_REMA_index_list.append(i)
    
print(CS2_REMA_z_list)

In [ ]:
dis_df = pd.DataFrame() #Create a dataframe of the distances
dis_df['Index'] = CS2_index_list
dis_df['Distance m'] = CS2_list
# dis_df['REMA Index'] = CS2_REMA_index_list
dis_df['REMA elevation'] = CS2_REMA_z_list

CS2_complete_df = CS2_df.join(dis_df.set_index('Index')) #join the original dataframe on the distances
print(CS2_complete_df.head)
print(CS2_complete_df.dtypes)

In [ ]:
cols = CS2_complete_df.select_dtypes(exclude=['float']).columns

CS2_complete_df[cols] = CS2_complete_df[cols].apply(pd.to_numeric, downcast='float', errors='coerce')


CS2_complete_df['REMA elevation'].astype('float64')

print(CS2_complete_df)
print(CS2_complete_df.dtypes)

In [ ]:
# Now remove rows where REMA elevation = NaN due to tile not spanning this distance
CS2_complete_df = CS2_complete_df.dropna(subset = ["REMA elevation"])          # Apply dropna() function
print(CS2_complete_df)  

CS2_complete_df = CS2_complete_df.drop([230,231,232])

In [ ]:
# plot where CS2 trackline is geographically

fig, (ax1) = plt.subplots(figsize=(16,16))

ax1.imshow(adjusted, cmap= custom_ramp,extent=[1980000.0, 2100000.0,-600000.0,-500000.0],origin='lower');
ax1=plt.gca(); ax1.ticklabel_format(useOffset=False, style='plain') #do not use scientific notation
rotatexlabels = plt.setp(ax1.get_xticklabels(),rotation=90) #rotate x tick labels 90 degrees
ax1.plot(CS2_df_v2['xx'],CS2_df_v2['yy'], linewidth=2, color ='firebrick', label='CS2 transect')
ax1.plot(D6['x'],D6['y'], linewidth=2, color ='blue', label='IS2transect')

ax1.axis('image')
ax1.set_xlim(1980000.0, 2100000.0,)
ax1.set_ylim(-600000.0,-500000.0)
# ax2.legend(loc="upper right")
ax1.set_xlabel('Longitude')
ax1.set_ylabel('Latitude')

plt.savefig('./Site D/CryoSat-2 images/CryoSat-2 Data map.png')
plt.show()

In [ ]:
# Now lets find elevation diffrence between REMA & CryoSat-2
CS2_elev1_vs_REMA = CS2_complete_df['CS2_elev_1'] - CS2_complete_df['REMA elevation']
print(CS2_elev1_vs_REMA)

# CS2_elev3_vs_REMA = CS2_complete_df['CS2_elev_3'] - CS2_complete_df['REMA elevation']
# print(CS2_elev3_vs_REMA)

# Basic Maths
# Mean min max

print('CS2_elev1 max: ', np.max(CS2_elev1_vs_REMA))
print('CS2_elev1 min: ', np.min(CS2_elev1_vs_REMA))
print('CS2_elev1 mean: ', np.mean(CS2_elev1_vs_REMA))

# print('CS2_elev3 max: ', np.max(CS2_elev3_vs_REMA))
# print('CS2_elev3 min: ', np.min(CS2_elev3_vs_REMA))
# print('CS2_elev3 mean: ', np.mean(CS2_elev3_vs_REMA))

# SD
standard_deviation_h1 = np.std(CS2_elev1_vs_REMA)
print('CS2_elev1 SD: ', standard_deviation_h1)

# standard_deviation_h3 = np.std(CS2_elev3_vs_REMA)
# print('CS2_elev3 SD: ', standard_deviation_h3)

# SEM
from scipy.stats import sem
SEM_h1 = sem(CS2_elev1_vs_REMA)
print('CS2_elev1 SEM: ', SEM_h1)

# SEM_h3 = sem(CS2_elev3_vs_REMA)
# print('CS2_elev3 SEM: ', SEM_h1)

# Correlation coefficent
from scipy.stats import pearsonr
print('CS2_elev1 correl: ', pearsonr(CS2_complete_df['REMA elevation'], CS2_complete_df['CS2_elev_1']))

# print('CS2_elev3 correl: ', pearsonr(CS2_complete_df['REMA elevation'], CS2_complete_df['CS2_elev_3']))

# R2 value
from sklearn.metrics import r2_score
r2_h1 = r2_score(CS2_complete_df['REMA elevation'], CS2_complete_df['CS2_elev_1'])
print('CS2_elev1 r2: ',r2_h1)

In [ ]:
# Check varible dimensions match

print('Distance m: ',len(IS2_complete_df['Distance m']))
print('Distance m: ',np.max(IS2_complete_df['Distance m']))
print('IS2_elevation: ',len(IS2_complete_df['IS2_elevation']))
print('REMA elevation: ',len(IS2_complete_df['REMA elevation']))

# Lets also check min max elevations of altimeters and REMA for the next plot
print('max: ', np.max(IS2_complete_df['IS2_elevation']))
print('min: ', np.min(IS2_complete_df['IS2_elevation']))

print('max: ', np.max(IS2_complete_df['REMA elevation']))
print('min: ', np.min(IS2_complete_df['REMA elevation']))

In [ ]:
# Now lets find elevation diffrence between REMA & ICESat-2
IS2_vs_REMA = IS2_complete_df['IS2_elevation'] - IS2_complete_df['REMA elevation']
print(IS2_vs_REMA)

# Basic Maths
# Mean min max

print('max: ', np.max(IS2_vs_REMA))
print('min: ', np.min(IS2_vs_REMA))
print('mean: ', np.mean(IS2_vs_REMA))


# SD
standard_deviation = np.std(IS2_vs_REMA)
print('SD: ', standard_deviation)

# SEM
from scipy.stats import sem
SEM = sem(IS2_vs_REMA)
print('SEM: ', SEM)

# Correlation coefficent
from scipy.stats import pearsonr
print('correl: ', pearsonr(IS2_complete_df['REMA elevation'], IS2_complete_df['IS2_elevation']))

# R2 value
from sklearn.metrics import r2_score
r2 = r2_score(IS2_complete_df['REMA elevation'], IS2_complete_df['IS2_elevation'])
print('r2: ',r2)

In [ ]:
# Final plot showing both x,y plots of REMA and altimeters and geographical plot with transect flightpath on
fig = plt.subplots(figsize=(18,5))
gs = gridspec.GridSpec(1,2, width_ratios=[2.75,1])

# Elevation plot
ax1 = plt.subplot(gs[0])

ax1.plot(IS2_complete_df['Distance m'],IS2_complete_df['REMA elevation'], linestyle='dotted',color ='firebrick', label='IS2 REMA transect')
ax1.plot(CS2_complete_df['Distance m'],CS2_complete_df['REMA elevation'], linestyle='dotted',color ='firebrick', label='CS2 REMA transect')

# ICESat-2 data
ax1.plot(IS2_complete_df['Distance m'],IS2_complete_df['IS2_elevation'], color ='blue', label='IS2 ATL06')

# CryoSat-2 data
ax1.plot(CS2_complete_df['Distance m'],CS2_complete_df['CS2_elev_1'], color ='orange', label='CS2 elev 1')

ax1.set_xlabel('Distance along flightpath transect (m)')
ax1.set_ylabel('Elevation (m)')
# ax1.set_xlim(0,102505)
ax1.legend(loc="upper right")

ax2 = plt.subplot(gs[1])
ax2.imshow(adjusted, cmap= custom_ramp,extent=[1980000.0, 2100000.0,-600000.0,-500000.0],origin='lower');
ax2=plt.gca(); ax2.ticklabel_format(useOffset=False, style='plain') #do not use scientific notation
rotatexlabels = plt.setp(ax2.get_xticklabels(),rotation=90) #rotate x tick labels 90 degrees
ax2.plot(D6['x'],D6['y'], linewidth=2, color ='blue', label='IS2 transect')

# CryoSat-2 line
ax2.plot(CS2_df_v2['xx'],CS2_df_v2['yy'], linewidth=2, color ='green', label='CS2 transect')

ax2.axis('image')
ax2.set_xlim(1980000.0, 2100000.0,)
ax2.set_ylim(-600000.0,-500000.0)
# ax2.legend(loc="upper right")
ax2.set_xlabel('Longitude')
ax2.set_ylabel('Latitude')

ax2.legend(loc="upper right")

plt.tight_layout()
plt.savefig('./Site D REMA transect fig.jpg',dpi=900, transparent=True)
plt.show()

In [ ]:
# Zoom test plot

# IS2 stats in MD extrcated region
 
df = IS2_complete_df.loc[IS2_complete_df['Distance m']>=2000]

df2 = df.loc[df['Distance m']<=5000,"IS2_elevation"]
#print(df2)

# iloc = 9549:9925



MD_IS2_subset_elev = SD_IS2_diff_df.iloc[4818:5001]#.values.tolist()
print(MD_IS2_subset_elev)
# #print(MD_IS2_subset_elev)
# MD_IS2_subset_sorted =[]
# for i in range(len(MD_IS2_subset_elev)):
#     a = str(MD_IS2_subset_elev[i])[1:-1]
#     MD_IS2_subset_sorted.append(a)
    
# MD_IS2_subset_sorted = list(np.float_(MD_IS2_subset_sorted))
# MD_IS2_subset_elev = MD_IS2_subset_sorted
# print(MD_IS2_subset_elev)


MD_IS2_subset_elev = MD_IS2_subset_elev['IS2']
print(MD_IS2_subset_elev)
print("")
print("Stats:")

# Mean min max

print('max: ', np.max(MD_IS2_subset_elev))
print('min: ', np.min(MD_IS2_subset_elev))
print('mean: ', np.mean(MD_IS2_subset_elev))

# SD
standard_deviation = np.std(MD_IS2_subset_elev)
print('SD: ', standard_deviation)

# SEM
from scipy.stats import sem
SEM = sem(MD_IS2_subset_elev)
print('SEM: ', SEM)

In [ ]:
from matplotlib.patches import Rectangle
# Zoom inset plot IS2
fig = plt.subplots(figsize=(18,22))
gs = gridspec.GridSpec(4,2, width_ratios=[2.5,1])

# Elevation plot
ax1 = plt.subplot(gs[2])

ax1.plot(IS2_complete_df['Distance m'],IS2_complete_df['REMA elevation'], linestyle='dotted',color ='firebrick', label='IS2 REMA elev')

# ICESat-2 data
ax1.plot(IS2_complete_df['Distance m'],IS2_complete_df['IS2_elevation'], color ='blue', label='IS2 ATL06')
ax1.set_xlabel('Distance along flightpath transect (m)', fontsize=20)
ax1.set_ylabel('Elevation (m)', fontsize=20)
ax1.set_xlim(0,)
ax1.legend(loc="upper right", fontsize=16)

ax3 = plt.subplot(gs[4])
# Zoom inset
axins = ax.inset_axes([-0.2, -0.1, 0.75, 1])  # set zoom axes
ax3.plot(IS2_complete_df['Distance m'],IS2_complete_df['REMA elevation'], linestyle='dotted',color ='firebrick', label='IS2 REMA elev')
ax3.plot(IS2_complete_df['Distance m'],IS2_complete_df['IS2_elevation'], color ='blue', label='IS2 ATL06')
ax3.set_xlabel('Distance along flightpath transect (m)', fontsize=20)
ax3.set_ylabel('Elevation (m)', fontsize=20)

x1, x2, y1, y2 = 2000,5000,2810,2840  # Set bounding area of data you would like to see zoomed in 
ax3.set_xlim(x1, x2)
ax3.set_ylim(y1, y2)
ax1.indicate_inset_zoom(ax3, edgecolor="black")

ax2 = plt.subplot(gs[3])
ax2.imshow(adjusted, cmap= custom_ramp,extent=[1980000.0, 2100000.0,-600000.0,-500000.0],origin='lower');
ax2=plt.gca(); ax2.ticklabel_format(useOffset=False, style='plain') #do not use scientific notation
rotatexlabels = plt.setp(ax2.get_xticklabels(),rotation=90) #rotate x tick labels 90 degrees
ax2.plot(D6['x'],D6['y'], linewidth=2, color ='Blue', label='IS2 transect')
ax2.axis('image')
ax2.legend(loc="lower right", fontsize=16)
ax2.set_xlabel('Longitude', fontsize=20)
ax2.set_ylabel('Latitude', fontsize=20)
ax2.set_xlim(1980000.0, 2100000.0,)
ax2.set_ylim(-600000.0,-500000.0)

ax4 = plt.subplot(gs[0])
ax4.plot(IS2_complete_df['Distance m'], SD_IS2_diff_df['IS2'], color ='blue', label='IS2 vs REMA')
ax4.set_xlim(0,)
ax4.set_ylabel('Elevation difference (m)', fontsize=20)
ax4.set_xlabel('Distance along flightpath transect (m)', fontsize=20)

ax5 = plt.subplot(gs[6])
ax5.plot(IS2_complete_df['Distance m'], SD_IS2_diff_df['IS2'], color ='blue', label='IS2 vs REMA')
ax5.plot([2000,5000], [0,0],color ='black', linewidth=1)
ax5.set_xlim(2000,5000)
ax5.set_ylabel('Elevation difference (m)', fontsize=20)
ax5.set_xlabel('Distance along flightpath transect (m)', fontsize=20)

ax1.tick_params(axis='both', which='both', labelsize=18) 
ax2.tick_params(axis='both', which='both', labelsize=18) 
ax3.tick_params(axis='both', which='both', labelsize=18) 
ax4.tick_params(axis='both', which='both', labelsize=18) 
ax5.tick_params(axis='both', which='both', labelsize=18)

ax6 = plt.subplot(gs[5])

sub_min_c = f"min:{np.min(MD_IS2_subset_elev)}"
sub_max_c = f"max:{np.max(MD_IS2_subset_elev)}"
sub_mean_c = f"mean:{np.mean(MD_IS2_subset_elev)}"
sub_SEM_c = f"SEM:{sem(MD_IS2_subset_elev)}"
sub_correl_c = f"R2:{r2}"

#plt.scatter(x=[1],y=[1], color='white')
extra = Rectangle((0,0), 1,1,fc="w",fill=False,edgecolor='none',linewidth=0)
legend2 = ax6.legend([extra,extra,extra,extra],[sub_min_c,sub_max_c,sub_mean_c,sub_SEM_c], loc='center left',bbox_to_anchor=(-0.3, 0.5), title ='Stats', edgecolor='none',fontsize=20)
plt.setp(legend2.get_title(),fontsize='24')
ax6.axis("off")

plt.tight_layout()
plt.savefig('./Site D zoom IS2 REMA transect fig.jpg',dpi=400, transparent=True)
plt.show()

In [ ]:
# CS2 stats in MD extrcated region
 
df = CS2_complete_df.loc[CS2_complete_df['Distance m']>=67500]

df2 = df.loc[df['Distance m']<=72500]
print(df2)

# iloc = 2580:2603


MD_CS2_subset_elev = SD_CS2_diff_df.iloc[102:117]
#print(MD_IS2_subset_elev)

elev_1_diff_df = pd.DataFrame(CS2_elev1_vs_REMA, columns=['Elev_1_diff'])


MD_CS2_subset_elev_1 = elev_1_diff_df.loc[102:117].values.tolist()
#print(MD_CS2_subset_elev_1)

MD_CS2_subset_sorted =[]
for i in range(len(MD_CS2_subset_elev_1)):
    a = str(MD_CS2_subset_elev_1[i])[1:-1]
    MD_CS2_subset_sorted.append(a)
    
MD_CS2_subset_sorted = list(np.float_(MD_CS2_subset_sorted))
MD_CS2_subset_elev_1 = MD_CS2_subset_sorted
print(MD_CS2_subset_elev_1)


In [ ]:
# Zoom inset plot CS2
fig = plt.subplots(figsize=(18,22))
gs = gridspec.GridSpec(4,2, width_ratios=[2.5,1])

# Elevation plot
ax1 = plt.subplot(gs[2])

ax1.plot(CS2_complete_df['Distance m'],CS2_complete_df['REMA elevation'], linestyle='dotted',color ='firebrick', label='CS2 REMA elev')

# CryoSat-2 data
ax1.plot(CS2_complete_df['Distance m'],CS2_complete_df['CS2_elev_1'], color ='orange', label='CS2 elev_1')
ax1.set_xlabel('Distance along flightpath transect (m)', fontsize=20)
ax1.set_ylabel('Elevation (m)', fontsize=20)
ax1.set_xlim(0,107000)
ax1.set_ylim(2600,2850)
ax1.legend(loc="upper right", fontsize=16)

ax3 = plt.subplot(gs[4])
# Zoom inset
# CryoSat-2 data
ax3.plot(CS2_complete_df['Distance m'],CS2_complete_df['CS2_elev_1'], color ='orange', label='CS2 elev_1')
ax3.plot(CS2_complete_df['Distance m'],CS2_complete_df['REMA elevation'], linestyle='dotted',color ='firebrick', label='CS2 REMA elev')
ax3.set_xlabel('Distance along flightpath transect (m)', fontsize=20)
ax3.set_ylabel('Elevation (m)', fontsize=20)

axins = ax.inset_axes([-0.2, -0.1, 0.75, 1])  # set zoom axes

x1, x2, y1, y2 = 67500,72500,2735,2770  # Set bounding area of data you would like to see zoomed in 
ax3.set_xlim(x1, x2)
ax3.set_ylim(y1, y2)
ax1.indicate_inset_zoom(ax3, edgecolor="black")


ax2 = plt.subplot(gs[3])
ax2.imshow(adjusted, cmap= custom_ramp,extent=[2000000.0, 2100000.0,-600000.0,-500000.0], origin='lower');
ax2=plt.gca(); ax2.ticklabel_format(useOffset=False, style='plain') #do not use scientific notation
rotatexlabels = plt.setp(ax2.get_xticklabels(),rotation=90) #rotate x tick labels 90 degrees
#ax2.plot(D6['x'],D6['y'], linewidth=2, color ='Blue', label='IS2 transect')
ax2.plot(CS2_df_v2['xx'],CS2_df_v2['yy'], linewidth=2, color ='green', label='CS2 transect')
ax2.axis('image')
ax2.legend(loc="upper right", fontsize=16)
ax2.set_xlabel('Longitude', fontsize=20)
ax2.set_ylabel('Latitude', fontsize=20)
ax2.set_xlim(2000000.0, 2100000.0)
ax2.set_ylim(-600000.0,-500000.0)


ax4 = plt.subplot(gs[0])
ax4.plot(CS2_complete_df['Distance m'], SD_CS2_diff_df['CS2_1'], color ='orange', label='CS2 elev1 vs REMA')
ax4.set_xlim(0,107000)
ax4.set_ylabel('Elevation difference (m)', fontsize=20)
ax4.set_xlabel('Distance along flightpath transect (m)', fontsize=20)

ax5 = plt.subplot(gs[6])
ax5.plot(CS2_complete_df['Distance m'], SD_CS2_diff_df['CS2_1'], color ='orange', label='CS2 elev1 vs REMA')
ax5.plot([67500,72500], [0,0],color ='black', linewidth=1)
ax5.set_xlim(67500,72500)
ax5.set_ylabel('Elevation difference (m)', fontsize=20)
ax5.set_xlabel('Distance along flightpath transect (m)', fontsize=20)

ax1.tick_params(axis='both', which='both', labelsize=18) 
ax2.tick_params(axis='both', which='both', labelsize=18) 
ax3.tick_params(axis='both', which='both', labelsize=18) 
ax4.tick_params(axis='both', which='both', labelsize=18) 
ax5.tick_params(axis='both', which='both', labelsize=18)

ax6 = plt.subplot(gs[5])

sub_title_1 = f"CS2 elev 1"
sub_min_c = f"min: {np.min(MD_CS2_subset_elev_1)}"
sub_max_c = f"max: {np.max(MD_CS2_subset_elev_1)}"
sub_mean_c = f"mean: {np.mean(MD_CS2_subset_elev_1)}"
sub_SEM_c = f"SEM: {sem(MD_CS2_subset_elev_1)}"
sub_correl_c = f"R2:{r2}"

#plt.scatter(x=[1],y=[1], color='white')
extra = Rectangle((0,0), 1,1,fc="w",fill=False,edgecolor='none',linewidth=0)
legend2 = ax6.legend([extra,extra,extra,extra,extra,extra,extra,extra,extra,extra,extra],[sub_title_1,sub_min_c,sub_max_c,sub_mean_c,sub_SEM_c, SPACE,
                                                                                          ],
                     loc='center left',bbox_to_anchor=(-0.3, 0.5), title ='Stats', edgecolor='none',fontsize=18)

plt.setp(legend2.get_title(),fontsize='22')
ax6.axis("off")

plt.tight_layout()
plt.savefig('./Site D zoom CS2 REMA transect fig.jpg',dpi=400, transparent=True)
plt.show()